# 13-03: 审计日志与沙箱安全安全机制包括审计日志、沙箱隔离和设备配对认证。

In [ ]:
// ========== 1. 审计日志系统 ==========
import { createWriteStream } from 'fs';
import { format } from 'util';
interface AuditLogEntry {
  timestamp: Date;
  level: 'info' | 'warning' | 'error' | 'critical';
  action: string;
  userId: string;
  resource: string;
  details: Record<string, unknown>;
  ipAddress?: string;
  userAgent?: string;
  success: boolean;
  errorMessage?: string;
}
class AuditLogger {
  private logStream: ReturnType<typeof createWriteStream>;
  private buffer: AuditLogEntry[] = [];
  private flushInterval: NodeJS.Timeout;

  constructor(logFile: string) {
    this.logStream = createWriteStream(logFile, { flags: 'a' });
    this.flushInterval = setInterval(() => this.flush(), 5000);
  }

  log(entry: Omit<AuditLogEntry, 'timestamp'>): void {
    const fullEntry: AuditLogEntry = {
      ...entry,
      timestamp: new Date()
    };

    this.buffer.push(fullEntry);

    // 严重错误立即写入
    if (entry.level === 'critical') {
      this.flush();
    }

    // 缓冲区满也写入
    if (this.buffer.length >= 100) {
      this.flush();
    }
  }

  private flush(): void {
    if (this.buffer.length === 0) return;

    const lines = this.buffer.map(e => JSON.stringify(e)).join('\n') + '\n';
    this.logStream.write(lines);
    this.buffer = [];
  }

  async query(filters: {
    startTime?: Date;
    endTime?: Date;
    userId?: string;
    action?: string;
    level?: AuditLogEntry['level'];
  }): Promise<AuditLogEntry[]> {
    // 实际实现需要读取日志文件并过滤
    // 这里简化处理
    return [];
  }

  close(): void {
    clearInterval(this.flushInterval);
    this.flush();
    this.logStream.end();
  }
}
// 使用示例
const auditLogger = new AuditLogger('audit.log');

auditLogger.log({
  level: 'info',
  action: 'USER_LOGIN',
  userId: 'user-123',
  resource: '/api/auth/login',
  details: { method: 'POST' },
  success: true
});

In [ ]:
// ========== 2. 沙箱隔离 ==========
import { spawn } from 'child_process';
import { mkdtempSync, writeFileSync, rmSync } from 'fs';
import { tmpdir } from 'os';
import { join } from 'path';
interface SandboxOptions {
  memoryLimitMB: number;
  timeoutMs: number;
  allowedModules?: string[];
  environment?: Record<string, string>;
}
interface SandboxResult {
  success: boolean;
  stdout: string;
  stderr: string;
  exitCode: number;
  executionTime: number;
}
class CodeSandbox {
  private options: SandboxOptions;

  constructor(options: Partial<SandboxOptions> = {}) {
    this.options = {
      memoryLimitMB: 128,
      timeoutMs: 30000,
      allowedModules: [],
      environment: {},
      ...options
    };
  }

  async execute(code: string): Promise<SandboxResult> {
    const tempDir = mkdtempSync(join(tmpdir(), 'sandbox-'));
    const codeFile = join(tempDir, 'script.js');

    // 包装代码以限制功能
    const wrappedCode = this.wrapCode(code);
    writeFileSync(codeFile, wrappedCode);

    const startTime = Date.now();

    return new Promise((resolve) => {
      const child = spawn('node', [
        `--max-old-space-size=${this.options.memoryLimitMB}`,
        '--experimental-vm-modules',
        codeFile
      ], {
        env: { ...process.env, ...this.options.environment },
        timeout: this.options.timeoutMs,
        killSignal: 'SIGTERM'
      });

      let stdout = '';
      let stderr = '';

      child.stdout.on('data', (data) => {
        stdout += data.toString();
      });

      child.stderr.on('data', (data) => {
        stderr += data.toString();
      });

      child.on('close', (exitCode) => {
        // 清理临时文件
        try {
          rmSync(tempDir, { recursive: true });
        } catch {}

        resolve({
          success: exitCode === 0,
          stdout,
          stderr,
          exitCode: exitCode || 0,
          executionTime: Date.now() - startTime
        });
      });

      child.on('error', (error) => {
        resolve({
          success: false,
          stdout,
          stderr: error.message,
          exitCode: -1,
          executionTime: Date.now() - startTime
        });
      });
    });
  }

  private wrapCode(code: string): string {
    return `
      // 禁用危险的 API
      delete require('child_process');
      process.exit = () => { throw new Error('exit() is disabled'); };

      // 设置资源限制
      const startTime = Date.now();
      setInterval(() => {
        if (Date.now() - startTime > ${this.options.timeoutMs}) {
          throw new Error('Execution timeout');
        }
      }, 1000);

      try {
        ${code}
      } catch (error) {
        console.error('Sandbox error:', error);
        process.exit(1);
      }
    `;
  }
}
// 使用示例
const sandbox = new CodeSandbox({
  memoryLimitMB: 64,
  timeoutMs: 5000
});

// sandbox.execute('console.log(1 + 1)');

In [ ]:
// ========== 3. 设备配对认证 ==========
import { randomBytes, createHash, createCipheriv, createDecipheriv } from 'crypto';
interface Device {
  id: string;
  name: string;
  publicKey: string;
  pairedAt: Date;
  lastSeen: Date;
  isTrusted: boolean;
}
interface PairingRequest {
  deviceId: string;
  deviceName: string;
  publicKey: string;
  pairingCode: string;
  expiresAt: Date;
}
class DevicePairing {
  private devices: Map<string, Device> = new Map();
  private pairingRequests: Map<string, PairingRequest> = new Map();
  private encryptionKey: Buffer;

  constructor() {
    // 32字节密钥用于 AES-256
    this.encryptionKey = randomBytes(32);
  }

  // 生成配对码
  generatePairingCode(deviceId: string, deviceName: string, publicKey: string): string {
    const pairingCode = randomBytes(4).toString('hex').toUpperCase(); // 8位码

    const request: PairingRequest = {
      deviceId,
      deviceName,
      publicKey,
      pairingCode,
      expiresAt: new Date(Date.now() + 10 * 60 * 1000) // 10分钟有效
    };

    this.pairingRequests.set(pairingCode, request);
    return pairingCode;
  }

  // 确认配对
  confirmPairing(pairingCode: string): Device | null {
    const request = this.pairingRequests.get(pairingCode);

    if (!request) {
      throw new Error('Invalid pairing code');
    }

    if (new Date() > request.expiresAt) {
      this.pairingRequests.delete(pairingCode);
      throw new Error('Pairing code expired');
    }

    const device: Device = {
      id: request.deviceId,
      name: request.deviceName,
      publicKey: request.publicKey,
      pairedAt: new Date(),
      lastSeen: new Date(),
      isTrusted: true
    };

    this.devices.set(device.id, device);
    this.pairingRequests.delete(pairingCode);

    return device;
  }

  // 验证设备
  verifyDevice(deviceId: string, signature: string, data: string): boolean {
    const device = this.devices.get(deviceId);
    if (!device || !device.isTrusted) {
      return false;
    }

    // 验证签名 (简化版)
    const expectedSignature = createHash('sha256')
      .update(data + device.publicKey)
      .digest('hex');

    if (signature !== expectedSignature) {
      return false;
    }

    device.lastSeen = new Date();
    return true;
  }

  // 撤销配对
  revokeDevice(deviceId: string): boolean {
    return this.devices.delete(deviceId);
  }

  // 加密通信数据
  encrypt(data: string, deviceId: string): string {
    const iv = randomBytes(16);
    const cipher = createCipheriv('aes-256-gcm', this.encryptionKey, iv);

    let encrypted = cipher.update(data, 'utf8', 'hex');
    encrypted += cipher.final('hex');

    const authTag = cipher.getAuthTag();

    return iv.toString('hex') + ':' + authTag.toString('hex') + ':' + encrypted;
  }

  // 解密通信数据
  decrypt(encryptedData: string): string {
    const [ivHex, authTagHex, encrypted] = encryptedData.split(':');

    const decipher = createDecipheriv(
      'aes-256-gcm',
      this.encryptionKey,
      Buffer.from(ivHex, 'hex')
    );

    decipher.setAuthTag(Buffer.from(authTagHex, 'hex'));

    let decrypted = decipher.update(encrypted, 'hex', 'utf8');
    decrypted += decipher.final('utf8');

    return decrypted;
  }

  getPairedDevices(): Device[] {
    return Array.from(this.devices.values());
  }
}
// 使用示例
const pairing = new DevicePairing();

// 新设备请求配对
const code = pairing.generatePairingCode(
  'device-001',
  'My Laptop',
  'public-key-xyz'
);
console.log('Pairing code:', code);

// 用户在主设备上输入配对码确认
const device = pairing.confirmPairing(code);
console.log('Device paired:', device?.name);